# Postprocessing of the KTH Actions dataset

This script is used AFTER training all of the models. Once this is done, we can load all of them for inference and get all of the correctly classified test samples from the UCF11-16f dataset.

Once this is done, we can then find the intersection of ALL of the models and randomize the ordering of this to generate a randomized dataset of correctly classified test samples for all models that can be used for verification.

In [ ]:
# Configuration - Docker container paths
BASE_DIR = "/nnv/code/nnv/examples/Submission/FORMALISE2025"
DATA_DIR = f"{BASE_DIR}/data"
MODELS_DIR = f"{BASE_DIR}/models"

In [ ]:
! pip install onnxruntime

In [ ]:
import numpy as np
import os
import onnxruntime as rt

In [ ]:
dir = BASE_DIR

In [ ]:
FRAMES_PER_VIDEO = 32

# load the data
data = np.load(os.path.join(DATA_DIR, 'KTHActions', 'grayscale', str(FRAMES_PER_VIDEO), f'kthactions_grayscale_{FRAMES_PER_VIDEO}f_test_data.npy'))
labels = np.load(os.path.join(DATA_DIR, 'KTHActions', 'grayscale', str(FRAMES_PER_VIDEO), f'kthactions_test_labels_{FRAMES_PER_VIDEO}f.npy'))

In [ ]:
data = data.astype(np.float32)

In [ ]:
def get_correct_indices(session):
  correct_indices = []
  input_name = session.get_inputs()[0].name

  for i in range(len(data)):
    sample = data[i]
    sample = np.expand_dims(np.expand_dims(sample, 0), 0)

    output_name = session.get_outputs()[0].name
    predictions = session.run([output_name], {input_name: sample})
    predicted = np.argmax(predictions[0])

    if predicted == labels[i]:
      correct_indices.append(i)

  return correct_indices

In [ ]:
# load the model
session = rt.InferenceSession(os.path.join(MODELS_DIR, f'kthactions_c3d_{FRAMES_PER_VIDEO}f.onnx'))
correct_indices = get_correct_indices(session)
print('done!')

In [ ]:
print(len(correct_indices))

In [ ]:
# randomize the indices to shuffle the test data randomly
import random
random.seed(42)
random.shuffle(correct_indices)

In [ ]:
correct_test_data = data[correct_indices]
correct_test_labels = labels[correct_indices]

# cut down to 100 samples
correct_test_data = correct_test_data[:100]
correct_test_labels = correct_test_labels[:100]

In [ ]:
# Save verification data - must match paths expected by verifykthactions.m
np.save(os.path.join(DATA_DIR, 'KTHActions', f'kthactions_grayscale_{FRAMES_PER_VIDEO}f_verification_data.npy'), correct_test_data)
np.save(os.path.join(DATA_DIR, 'KTHActions', f'kthactions_grayscale_{FRAMES_PER_VIDEO}f_verification_labels.npy'), correct_test_labels)